In [43]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [44]:
#path = "../ml-20m/"
path = "/home/garvit/Desktop/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [3]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [4]:
len(ratings)

100004

In [5]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

user id to idx 

In [8]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [9]:
print(ratings.movieId)


0            0
1            1
2            2
3            3
4            4
5            5
6            6
7            7
8            8
9            9
10          10
11          11
12          12
13          13
14          14
15          15
16          16
17          17
18          18
19          19
20          20
21          21
22          22
23          23
24          24
25          25
26          26
27          27
28          28
29          29
          ... 
99974      473
99975      354
99976      355
99977     5577
99978      477
99979      478
99980      358
99981      479
99982      480
99983      359
99984     1225
99985     1240
99986      361
99987      126
99988     1260
99989      483
99990      362
99991      127
99992      364
99993     1299
99994      412
99995      486
99996     1308
99997      365
99998     2930
99999     7005
100000    4771
100001    1329
100002    1331
100003    2946
Name: movieId, Length: 100004, dtype: int64


In [10]:
print (ratings.userId)

0           0
1           0
2           0
3           0
4           0
5           0
6           0
7           0
8           0
9           0
10          0
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
20          1
21          1
22          1
23          1
24          1
25          1
26          1
27          1
28          1
29          1
         ... 
99974     670
99975     670
99976     670
99977     670
99978     670
99979     670
99980     670
99981     670
99982     670
99983     670
99984     670
99985     670
99986     670
99987     670
99988     670
99989     670
99990     670
99991     670
99992     670
99993     670
99994     670
99995     670
99996     670
99997     670
99998     670
99999     670
100000    670
100001    670
100002    670
100003    670
Name: userId, Length: 100004, dtype: int64


In [11]:
print(ratings.head)

<bound method DataFrame.head of         userId  movieId  rating   timestamp
0            0        0     2.5  1260759144
1            0        1     3.0  1260759179
2            0        2     3.0  1260759182
3            0        3     2.0  1260759185
4            0        4     4.0  1260759205
5            0        5     2.0  1260759151
6            0        6     2.0  1260759187
7            0        7     2.0  1260759148
8            0        8     3.5  1260759125
9            0        9     2.0  1260759131
10           0       10     2.5  1260759135
11           0       11     1.0  1260759203
12           0       12     4.0  1260759191
13           0       13     4.0  1260759139
14           0       14     3.0  1260759194
15           0       15     2.0  1260759198
16           0       16     2.0  1260759108
17           0       17     2.5  1260759113
18           0       18     1.0  1260759200
19           0       19     3.0  1260759117
20           1       20     4.0   835355493


In [12]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [13]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [14]:
n_factors = 50

In [15]:
np.random.seed = 42

In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [17]:
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [18]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=gli.sort_values(ascending=False)[:15]

In [19]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [20]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [21]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


In [22]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [23]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [24]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/1
80028/80028 [==============================] - 5s - loss: 9.9771 - val_loss: 4.3215


In [25]:
model.optimizer.lr=0.01

In [26]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/3
80028/80028 [==============================] - 8s - loss: 3.1580 - val_loss: 2.8258
Epoch 2/3
80028/80028 [==============================] - 7s - loss: 2.4084 - val_loss: 2.6148
Epoch 3/3
80028/80028 [==============================] - 7s - loss: 2.2263 - val_loss: 2.5629


In [27]:
model.optimizer.lr=0.001

In [28]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/6
80028/80028 [==============================] - 7s - loss: 2.1543 - val_loss: 2.5538
Epoch 2/6
80028/80028 [==============================] - 7s - loss: 2.1146 - val_loss: 2.5515
Epoch 3/6
80028/80028 [==============================] - 7s - loss: 2.0886 - val_loss: 2.5603
Epoch 4/6
80028/80028 [==============================] - 7s - loss: 2.0690 - val_loss: 2.5660
Epoch 5/6
80028/80028 [==============================] - 7s - loss: 2.0509 - val_loss: 2.5657
Epoch 6/6
80028/80028 [==============================] - 7s - loss: 2.0327 - val_loss: 2.5752


In [29]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [30]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [31]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [32]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [33]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [34]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/1
80028/80028 [==============================] - 5s - loss: 8.9724 - val_loss: 3.5567


In [35]:
model.optimizer.lr=0.01

In [36]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/6
80028/80028 [==============================] - 8s - loss: 2.5989 - val_loss: 2.3114
Epoch 2/6
80028/80028 [==============================] - 7s - loss: 1.9968 - val_loss: 2.1075
Epoch 3/6
80028/80028 [==============================] - 7s - loss: 1.8345 - val_loss: 2.0185
Epoch 4/6
80028/80028 [==============================] - 7s - loss: 1.7401 - val_loss: 1.9446
Epoch 5/6
80028/80028 [==============================] - 7s - loss: 1.6608 - val_loss: 1.8765
Epoch 6/6
80028/80028 [==============================] - 7s - loss: 1.5863 - val_loss: 1.8044


In [37]:
model.optimizer.lr=0.001

In [38]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/10
80028/80028 [==============================] - 7s - loss: 1.5119 - val_loss: 1.7439
Epoch 2/10
80028/80028 [==============================] - 7s - loss: 1.4411 - val_loss: 1.6844
Epoch 3/10
80028/80028 [==============================] - 7s - loss: 1.3721 - val_loss: 1.6260
Epoch 4/10
80028/80028 [==============================] - 8s - loss: 1.3054 - val_loss: 1.5733
Epoch 5/10
80028/80028 [==============================] - 8s - loss: 1.2426 - val_loss: 1.5221
Epoch 6/10
80028/80028 [==============================] - 7s - loss: 1.1802 - val_loss: 1.4761
Epoch 7/10
80028/80028 [==============================] - 8s - loss: 1.1235 - val_loss: 1.4278
Epoch 8/10
80028/80028 [==============================] - 8s - loss: 1.0687 - val_loss: 1.3864
Epoch 9/10
80028/80028 [==============================] - 8s - loss: 1.0168 - val_loss: 1.3444
Epoch 10/10
80028/80028 [==============================] - 8s - loss: 0.9671 - val_loss: 1.3062

In [39]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80028 samples, validate on 19976 samples
Epoch 1/5
80028/80028 [==============================] - 8s - loss: 0.9214 - val_loss: 1.2747
Epoch 2/5
80028/80028 [==============================] - 8s - loss: 0.8779 - val_loss: 1.2412
Epoch 3/5
80028/80028 [==============================] - 8s - loss: 0.8377 - val_loss: 1.2121
Epoch 4/5
80028/80028 [==============================] - 8s - loss: 0.8005 - val_loss: 1.1856
Epoch 5/5
80028/80028 [==============================] - 8s - loss: 0.7666 - val_loss: 1.1627


This result is quite a bit better than the best benchmarks that we could find with a quick google search - so looks like a great approach!

In [40]:
model.save_weights(model_path+'bias.h5')

In [41]:
model.load_weights(model_path+'bias.h5')